This program works with triple binary array which have 3 columns and 3 rows .
I will more demonstrate the code in next cellules ..


In [ ]:
!pip install mpi4py
import numpy as np
import hashlib
from mpi4py import MPI
from itertools import permutations
import sys


row,col=4,4


comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

notSatisfyConstraint=True
while notSatisfyConstraint:
  rowSum = input('Enter sum of each row in wanted triple matrix (Should tap a space after every row sum): ')
  for item in rowSum.split():

    if item.strip().isdigit():
       notSatisfyConstraint=False
       if int(item)<0 or int(item)>3:
         print('\nAll rows sum should be bigger then or aqual to 0 and less then or equal to 3 ! Try again \n')
         notSatisfyConstraint=True
         break
    else:
          print('Should enter only integer values! Try again so \n')
          break

rowTuple = tuple(int(item) for item in rowSum.split())
print("rows sum tuple you choised :")
print(rowTuple)

notSatisfyConstraint=True
while notSatisfyConstraint:
  columnSum = input('Enter sum of each column in wanted triple matrix (Should tap a space after every column sum): ')
  for item in columnSum.split():

    if item.strip().isdigit():
       notSatisfyConstraint=False
       if int(item)<0 or int(item)>3:
         print('\nAll columns sum should be bigger then or aqual to 0 and less then or equal to 3 ! Try again \n')
         notSatisfyConstraint=True
         break
    else:
          print('Should enter only integer values! Try again so \n')
          break
columnTuple = tuple(int(item) for item in columnSum.split())
print("columns sum tuple you choised :")
print(columnTuple)

row,col=3,3
n = row
count = n // size
remainder = n % size

if rank < remainder:
    start = rank * (count + 1)
    stop = start + count + 1
else:
    start = rank * count + remainder
    stop = start + count

def rowSumCount(param1, param2, param3):
    r = param1+param2+param3
    return (r)

def columnSumCount(param1, param2, param3):
    c = param1+param2+param3
    return (c)





def countSumOfColumns(myArray):
  portionArray = myArray[:, start:stop]
  fArray = np.empty((portionArray.shape[0]+1, portionArray.shape[1]))
  fArray[:portionArray.shape[0], :] = portionArray
  fArray[-1, :] = columnSumCount(fArray[0, :], fArray[1, :], fArray[2, :])

  if rank > 0:
     comm.Send(fArray[-1, :], dest=0, tag=10)
  else:
    columnSumList=fArray[-1, :].copy()
    for i in range(1, size):
        if i < remainder:
            rank_size = count + 1
        else:
            rank_size = count
        tmp = []
        comm.Recv(tmp, source=i, tag=10)
        columnSumList.extend(tmp)
    columnSum=tuple(columnSumList)
    return (columnSum)



def countSumOfRows(myArray):
  portionArray = myArray[start:stop, :]
  fArray = np.empty((portionArray.shape[0], portionArray.shape[1] + 1))
  fArray[:, :portionArray.shape[1]] = portionArray
  fArray[:, -1] = rowSumCount(fArray[:, 0], fArray[:, 1], fArray[:, 2])

  if rank > 0:

    comm.Send(fArray[:, -1], dest=0, tag=14)

  else:
    rowSumList=fArray[:, -1].copy()
    for i in range(1, size):
        if i < remainder:
            rank_size = count + 1
        else:
            rank_size = count
        tmp = []
        comm.Recv(tmp, source=i, tag=14)

        rowSumList.extend(tmp)
    rowSum=tuple(rowSumList)
    return (rowSum)


def hashFor(array):
    hashId = hashlib.md5()
    hashId.update(repr(array).encode('utf-8'))
    return hashId.hexdigest()

existArray=True
while True:
      getArray = np.random.randint(2,size=(row,col))
      if countSumOfRows(getArray) == rowTuple and countSumOfColumns(getArray) == columnTuple :
         originalArray=getArray
         break
if not existArray:
       print("There is no array satisfies these rows sum and columns sum constraints you entered ! Try again")
       sys.exit()

print(f'The obtained Array ( Original array ):\n {str(originalArray)}' )
print(f'Original array hash : {str(hashFor(originalArray))}')


l, firstRow, secondRow ,thirdRow = ([] for i in range(4))

for rowRank,i in enumerate(rowTuple):

    for j in range (i):
        l.append(1)
    for j in range (3-i):
        l.append(0)
    if rowRank==0:
      firstRow.extend(l)
    elif rowRank==1:
      secondRow.extend(l)
    elif rowRank==2:
      thirdRow.extend(l)
    l.clear()




perm1 = permutations(firstRow)
perm2 = permutations(secondRow)
perm3 = permutations(thirdRow)

list1, list2, list3 ,List1 ,List2 ,List3= ([] for i in range(6))
for i,j,k in zip( list(perm1) ,list(perm2) ,list(perm3) ):
    list1.append(i)
    list2.append(j)
    list3.append(k)




for i,j,k in zip(list1 ,list2 ,list3):
    if i not in List1:
        List1.append(i)
    if j not in List2:
        List2.append(j)
    if k not in List3:
        List3.append(k)


print("The similar arrays found : ")
arrayList= []
for i in List1:
   arrayList.append(i)

   for j in List2:
       arrayList.append(j)

       for k in List3:
           arrayList.append(k)
           array=np.asarray(arrayList)
           if countSumOfRows(array) == rowTuple and countSumOfColumns(array) == columnTuple :
              print(array)
              if hashFor(array)==hashFor(originalArray):
                 similarArray=array
           del arrayList[-1]
       del arrayList[-1]
   del arrayList[-1]

print(f'The program finally found the original array :\n {str(similarArray)}' )


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Enter sum of each row in wanted triple matrix (Should tap a space after every row sum): 2 2 1
rows sum tuple you choised :
(2, 2, 1)
Enter sum of each column in wanted triple matrix (Should tap a space after every column sum): 2 2 1
columns sum tuple you choised :
(2, 2, 1)
The obtained Array ( Original array ):
 [[1 1 0]
 [0 1 1]
 [1 0 0]]
Original array hash : 30aa70582918ccfdab500f8472fba05c
The similar arrays found : 
[[1 1 0]
 [1 1 0]
 [0 0 1]]
[[1 1 0]
 [1 0 1]
 [0 1 0]]
[[1 1 0]
 [0 1 1]
 [1 0 0]]
[[1 0 1]
 [1 1 0]
 [0 1 0]]
[[0 1 1]
 [1 1 0]
 [1 0 0]]
The program finally found the original array :
 [[1 1 0]
 [0 1 1]
 [1 0 0]]


first, I used MPI4PY to get parallel program . COMM_WORLD gives access to the number of processes (ranks/processors) available to distribute work across, and information about each processor. size gives the total number of ranks, or processors, allocated to run our program. rank gives the identifier of the processor currently executing the code.
Then, the user enter values for each row sum , but must after every values should enter space , if one of values entered was not integer there will be error message which asks for user to reenter new value. one more error message will appears when one of values more then 3 or less then 0 entered.
The same thing about columns sum.


In [ ]:
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

notSatisfyConstraint=True
while notSatisfyConstraint:
  rowSum = input('Enter sum of each row in wanted triple matrix (Should tap a space after every row sum): ')
  for item in rowSum.split():

    if item.strip().isdigit():
       notSatisfyConstraint=False
       if int(item)<0 or int(item)>3:
         print('\nAll rows sum should be bigger then or aqual to 0 and less then or equal to 3 ! Try again \n')
         notSatisfyConstraint=True
         break
    else:
          print('Should enter only integer values! Try again so \n')
          break

rowTuple = tuple(int(item) for item in rowSum.split())
print("rows sum tuple you choised :")
print(rowTuple)

notSatisfyConstraint=True
while notSatisfyConstraint:
  columnSum = input('Enter sum of each column in wanted triple matrix (Should tap a space after every column sum): ')
  for item in columnSum.split():

    if item.strip().isdigit():
       notSatisfyConstraint=False
       if int(item)<0 or int(item)>3:
         print('\nAll columns sum should be bigger then or aqual to 0 and less then or equal to 3 ! Try again \n')
         notSatisfyConstraint=True
         break
    else:
          print('Should enter only integer values! Try again so \n')
          break
columnTuple = tuple(int(item) for item in columnSum.split())
print("columns sum tuple you choised :")
print(columnTuple)


Now, divide up the list of array, assigning a chunk of the array to each process (or rank). The number of rows in array (3) , so it can oddly divisible by the number of processors, so that we have to do a little extra math to break up . Now each processor has a variable indexing the start and stop locations of its chunk in the array.



In [ ]:
row,col=3,3
n = row
count = n // size
remainder = n % size

if rank < remainder:
    start = rank * (count + 1)
    stop = start + count + 1
else:
    start = rank * count + remainder
    stop = start + count



Here ,I defined 2 functions which first one add the values param1+param2+param3 wich next will be the value of each row in array , and the other the same operation but will applied with values of each rows.

In [ ]:
def rowSumCount(param1, param2, param3):
    r = param1+param2+param3
    return (r)

def columnSumCount(param1, param2, param3):
    c = param1+param2+param3
    return (c)

here , I used to final result to be an array with myArray values and function result for each array set. Create an empty array, fArray (finalArray) with the same number of rows as the original array and one extra column to store the results. Then run columnSumCount function for each array set and save the result in the result array (fArray).Now each processor has results for its chunk of the parms array.

then ,the sending of fArray arrays from each rank to rank ‘0’, where they are combined to a single array.I specify the rank to send to , dest, and a tag so the receiving rank knows which value to retrieve .

For the receiving rank (0),There is loop through all the other ranks, creating an empty list, and I retrieve the sent values from each rank with Recv, specifying the rank to receive from and the tag. Finally for each receiving value, it will be add in list columnSumList and .
The same thing with rowSumCount.

In [ ]:
def countSumOfColumns(myArray):
  portionArray = myArray[:, start:stop]
  fArray = np.empty((portionArray.shape[0]+1, portionArray.shape[1]))
  fArray[:portionArray.shape[0], :] = portionArray
  fArray[-1, :] = columnSumCount(fArray[0, :], fArray[1, :], fArray[2, :])

  if rank > 0:
     comm.Send(fArray[-1, :], dest=0, tag=10)
  else:
    columnSumList=fArray[-1, :].copy()
    for i in range(1, size):
        if i < remainder:
            rank_size = count + 1
        else:
            rank_size = count
        tmp = []
        comm.Recv(tmp, source=i, tag=10)
        columnSumList.extend(tmp)
    columnSum=tuple(columnSumList)
    return (columnSum)



def countSumOfRows(myArray):
  portionArray = myArray[start:stop, :]
  fArray = np.empty((portionArray.shape[0], portionArray.shape[1] + 1))
  fArray[:, :portionArray.shape[1]] = portionArray
  fArray[:, -1] = rowSumCount(fArray[:, 0], fArray[:, 1], fArray[:, 2])

  if rank > 0:

    comm.Send(fArray[:, -1], dest=0, tag=14)

  else:
    rowSumList=fArray[:, -1].copy()
    for i in range(1, size):
        if i < remainder:
            rank_size = count + 1
        else:
            rank_size = count
        tmp = []
        comm.Recv(tmp, source=i, tag=14)

        rowSumList.extend(tmp)
    rowSum=tuple(rowSumList)
    return (rowSum)



The difinition of array hash function.
Then, the program will search for a random triple binary array specifies rows sum and columns sum the user entered.

In [ ]:
def hashFor(array):
    hashId = hashlib.md5()
    hashId.update(repr(array).encode('utf-8'))
    return hashId.hexdigest()

existArray=True
while True:
      getArray = np.random.randint(2,size=(row,col))
      if countSumOfRows(getArray) == rowTuple and countSumOfColumns(getArray) == columnTuple :
         originalArray=getArray
         break
if not existArray:
       print("There is no array satisfies these rows sum and columns sum constraints you entered ! Try again")
       sys.exit()

print(f'The obtained Array ( Original array ):\n {str(originalArray)}' )
print(f'Original array hash : {str(hashFor(originalArray))}')


Try to construct lists wich all of them are one row of the original array ,
starting from the number of 1 in the row sum tuple. that means if we have 2, in first loop the programm fills in the list with 2 ones , then it stays one more place in list (because we have 3 items ) the program fills in it with zero.
for example when row sum tuple is (2, 2, 3)
the lists of array that will be constructed later :
[1, 1, 0]
[1, 1, 0]
[1, 1, 1]
but we will apply permutations with all of this lists (as rows of array)


In [ ]:
l, firstRow, secondRow ,thirdRow = ([] for i in range(4))

for rowRank,i in enumerate(rowTuple):

    for j in range (i):
        l.append(1)
    for j in range (3-i):
        l.append(0)
    if rowRank==0:
      firstRow.extend(l)
    elif rowRank==1:
      secondRow.extend(l)
    elif rowRank==2:
      thirdRow.extend(l)
    l.clear()



construct all permutations of every list we obtained (every row list above)

In [ ]:
perm1 = permutations(firstRow)
perm2 = permutations(secondRow)
perm3 = permutations(thirdRow)

list1, list2, list3 ,List1 ,List2 ,List3= ([] for i in range(6))
for i,j,k in zip( list(perm1) ,list(perm2) ,list(perm3) ):
    list1.append(i)
    list2.append(j)
    list3.append(k)



probably one of row sum values, if not all of them ,will be 2 for example ,that means the permutations of the list [1, 1, 0] will have 2 similar permutation ([1, 1, 0], [1, 1, 0]) with two values of 1 , but we should avoid the similar permutations , and for that I tried to create new row lists for applying permutation without repetition .


In [ ]:
for i,j,k in zip(list1 ,list2 ,list3):
    if i not in List1:
        List1.append(i)
    if j not in List2:
        List2.append(j)
    if k not in List3:
        List3.append(k)



the last part of this program represents the array wich we got it as a result of composition of each permutations of each list with all permutations of the other lists .Once we obtain an triple array ,we confirm if it has same rows sum and columns sum and print it as similar array , then we compare it's hash with the original array hash, if it is valid we can class it as the original array.

In [ ]:
print("The similar arrays found : ")
arrayList= []
for i in List1:
   arrayList.append(i)

   for j in List2:
       arrayList.append(j)

       for k in List3:
           arrayList.append(k)
           array=np.asarray(arrayList)
           if countSumOfRows(array) == rowTuple and countSumOfColumns(array) == columnTuple :
              print(array)
              if hashFor(array)==hashFor(originalArray):
                 similarArray=array
           del arrayList[-1]
       del arrayList[-1]
   del arrayList[-1]

print(f'The program finally found the original array :\n {str(similarArray)}' )